<a href="https://colab.research.google.com/github/AlenaKV/TestQA/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import os
import numpy as np
import math
import sqlite3
!pip install pyyaml
import yaml


data2 = {
    "metric": {
        "std": 10,
        "var": 10,
    }
}
with open("test.yaml", "w") as file:
    yaml.dump(data2, file)

with open("test.yaml") as file:
    data1 = yaml.safe_load(file)

print(data1)

img = cv2.imread("/content/test_picture3.jpg")
if img is None:
  print('Image not found')
else:
  gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # конвертация из серого в бинарное
  ret, thresh = cv2.threshold(gray_image, 127, 255, 0)
  # расчет моментов на бинарном изображении
  M = cv2.moments(thresh)
  #расчет коориднат центра объекта
  centrX = int(M["m10"] / M["m00"]) #координата Х центра
  centrY = int(M["m01"] / M["m00"]) #кооридната Y центра


  contours=cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  contours_p = np.vstack(contours[0]).squeeze()

  x_coordinates = contours_p[:, 0]
  y_coordinates = contours_p[:, 1]
  image = cv2.circle(img, (centrX,centrY), radius=0, color=(0, 0, 255), thickness=-1)
  cv2.imwrite("test_image_result3.png",image)
  print('Коорданата центра Х:', centrX)
  print('Координата центра Y:', centrY)

  lenghtvec=[]
  for i in range(len(x_coordinates)):
    lenghtvec.append(math.sqrt((x_coordinates[i]-centrX)**2+(y_coordinates[i]-centrY)**2))

  mean = np.mean(lenghtvec)
  print('Дисперсия: ', np.var(lenghtvec))
  print('Стандартное отклонение: ' ,np.std(lenghtvec))

  # Создаем подключение к базе данных
  connection = sqlite3.connect('database_metric.db')
  #Для отправки SQL-запросов в базу данных
  cursor = connection.cursor()

  # Создаем таблицу Metrics
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Metrics (
  id INTEGER PRIMARY KEY,
  variance REAL NOT NULL,
  standard_deviation REAL NOT NULL
  )
  ''')
  cursor.execute('INSERT INTO Metrics (variance, standard_deviation) VALUES (?, ?)', (np.var(lenghtvec), np.std(lenghtvec)))
  cursor.execute('SELECT * FROM Metrics')
  metrics_fromdb = cursor.fetchall()
  print(metrics_fromdb)

# Сохраняем изменения и закрываем соединение
  connection.commit()
  connection.close()

  print('_________________________________________')






{'metric': {'std': 10, 'var': 10}}
Коорданата центра Х: 103
Координата центра Y: 105
Дисперсия:  0.5760325591165127
Стандартное отклонение:  0.7589680883387079
[(1, 0.5760325591165127, 0.7589680883387079)]
_________________________________________
